```{contents}
```
## Cache Invalidation 


**Cache invalidation** is the process of **removing or refreshing cached data** when it becomes **stale, incorrect, or unsafe to reuse**.

Without invalidation, caches eventually return **wrong answers**.

---

### Why Cache Invalidation Is Critical

| Problem               | Impact            |
| --------------------- | ----------------- |
| Stale data            | Wrong outputs     |
| Model changes         | Incorrect reuse   |
| Knowledge updates     | Outdated facts    |
| Business rule updates | Policy violations |

---

### Invalidation in the Request Pipeline

```
Request
  ↓
Cache Check
  ↓ hit → Return Cached Data (only if still valid)
  ↓ invalid / miss
Recompute → Update Cache → Return
```

---

### Manual Invalidation

#### Demonstration

```python
cache = {}

def invalidate(key):
    cache.pop(key, None)
```

Use when:

* A specific record is updated
* A user changes settings
* Admin forces refresh

---

### Version-Based Invalidation (Most Reliable)

Changing the version automatically makes old cache unreachable.

#### Demonstration

```python
MODEL_VERSION = "v2"

def cache_key(prompt):
    return f"{MODEL_VERSION}:{prompt}"

cache = {}

def get_response(prompt):
    key = cache_key(prompt)

    if key in cache:
        return cache[key]

    response = llm.invoke(prompt).content
    cache[key] = response
    return response
```

Updating `MODEL_VERSION` invalidates **all previous cache entries** instantly.

---

### Time-Based Invalidation (TTL)

Data expires automatically after a fixed time.

#### Demonstration

```python
from cachetools import TTLCache

cache = TTLCache(maxsize=10000, ttl=3600)

def get_data(key):
    if key in cache:
        return cache[key]

    value = expensive_operation(key)
    cache[key] = value
    return value
```

---

### Event-Based Invalidation

Cache is cleared when specific system events occur.

#### Demonstration

```python
def on_knowledge_update():
    cache.clear()
```

Trigger on:

* Database updates
* Knowledge base refresh
* Configuration changes

---

### Selective Invalidation

Only related entries are removed.

#### Demonstration

```python
def invalidate_user(user_id):
    for key in list(cache.keys()):
        if key.startswith(f"user:{user_id}:"):
            cache.pop(key)
```

---

### Cache Invalidation Rules for AI Systems

| Trigger                | Invalidate                   |
| ---------------------- | ---------------------------- |
| Model change           | All responses                |
| Prompt template change | All related entries          |
| Knowledge update       | Retrieval cache              |
| User preference change | User scope cache             |
| Policy update          | All policy-dependent entries |

---

### Mental Model

```
Cache Invalidation = Garbage collection for correctness
```

---

### Key Takeaways

* Caching without invalidation causes silent failure
* Versioning is the safest strategy
* Combine TTL + versioning + event triggers
* Invalidation is the hardest part of caching systems